## Coletando dados de zonas e páginas do Mercado Livre e salvando em um dataframe

In [49]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [50]:
zonas = ['sul', 'norte', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']

In [51]:
url_ml = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

In [52]:
url_ml.format(zonas, paginas)

"https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-['sul', 'norte', 'leste', 'oeste']/['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']"

In [53]:
re_quarto = '\| (.*) quarto'
re_area = '> (.*) m²'

In [54]:
def coleta_dados(url, zona):
    
    conteudo = requests.get(url).content
    sopa = BeautifulSoup(conteudo)
    
    precos = sopa.find_all('span', class_='price__fraction')
    areas_e_quartos = sopa.find_all('div', class_='item__attrs')
        
    lista_areas = []
    lista_quartos = []
    lista_precos = []
    lista_zonas = []

    for padrao in precos:
        preco = re.findall('<span class="price__fraction">(.*)</span', str(padrao))
        lista_precos.append(preco[0].replace('.',''))
    
    
    for elemento in areas_e_quartos:    
        quarto = re.findall(re_quarto, str(elemento))
        area = re.findall(re_area, str(elemento))
        
        if quarto == []:
            quarto = np.nan
        else:
            quarto = quarto[0]
        
        if area == []:
            area = np.nan
        else:
            area=area[0]
            
            
        lista_quartos.append(quarto)
        lista_areas.append(area)                     
        lista_zonas.append(zona)        
        
    
    dados = {'zona': lista_zonas,
             'area': lista_areas,
             'preco': lista_precos,
             'quartos': lista_quartos}     

    df = pd.DataFrame(dados)    
    return df 

In [55]:
lista_dfs = []
for zona in zonas:
    for pagina in paginas:        
        url = url_ml.format(zona, pagina)
        df = coleta_dados(url,zona)
        print('Carregando a url', url)
        lista_dfs.append(df)   
        sleep(2)
print('Dados coletados!')

Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_241
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Carregando a url https://imoveis.mercadolivre.com.br/ca

In [57]:
dados = pd.concat(lista_dfs)

In [58]:
dados

,zona,area,preco,quartos
0,sul,300,10900,NaN
1,sul,250,4500,NaN
2,sul,130,2800,3
3,sul,135,4500,3
4,sul,300,12000,3
...,...,...,...,...
43,oeste,190,12000,2
44,oeste,212,6200,3
45,oeste,190,7300,4
46,oeste,425,26000,2


In [62]:
dados.to_csv('dados_mercado_livre.csv', index=False)

In [64]:
df = pd.read_csv('dados_mercado_livre.csv')

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 4 columns):
zona       720 non-null object
area       703 non-null float64
preco      720 non-null int64
quartos    638 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 22.6+ KB


In [66]:
df.count()

zona       720
area       703
preco      720
quartos    638
dtype: int64

In [68]:
df.isnull().sum()

zona        0
area       17
preco       0
quartos    82
dtype: int64

In [70]:
df.shape #Linha, #Colunas

(720, 4)